### For part 1, question 2

In [2]:
import numpy as np


In [1]:
from cvxopt import matrix, solvers

# Data points
X = np.array([[3, 3], [4, 3], [1, 1]])
y = np.array([1, 1, -1])

# Number of data points
n = X.shape[0]

# Compute the Gram matrix (K)
K = np.dot(X, X.T) * np.outer(y, y)

# Define the quadratic programming problem
P = matrix(K, tc='d')
q = matrix(-np.ones(n), tc='d')

# Non-negativity constraints (alpha_i >= 0)
G = matrix(-np.eye(n), tc='d')
h = matrix(np.zeros(n), tc='d')

# Equality constraints (sum(alpha_i * y_i) = 0)
A = matrix(y, (1, n), tc='d')
b = matrix(0.0, tc='d')

# Solve the quadratic programming problem
sol = solvers.qp(P, q, G, h, A, b)

# Extract the solution (alpha values)
alphas = np.array(sol['x']).flatten()

print("Optimal alpha values:", alphas)

     pcost       dcost       gap    pres   dres
 0: -2.6873e-01 -6.7301e-01  4e+00  2e+00  1e+00
 1: -1.4232e-01 -5.9278e-01  5e-01  3e-16  6e-16
 2: -2.3773e-01 -2.7513e-01  4e-02  6e-17  5e-16
 3: -2.4959e-01 -2.5037e-01  8e-04  6e-17  4e-16
 4: -2.5000e-01 -2.5000e-01  8e-06  6e-17  2e-16
 5: -2.5000e-01 -2.5000e-01  8e-08  3e-17  3e-16
Optimal solution found.
Optimal alpha values: [2.49999916e-01 8.17639192e-08 2.49999998e-01]


In [3]:
# given a basis matrix A_b find the inverse of A_b
def inverse(A_b):
    n = A_b.shape[0]
    I = np.eye(n)
    A_inv = np.zeros((n, n))
    for i in range(n):
        b = I[:, i]
        x = np.linalg.solve(A_b, b)
        A_inv[:, i] = x
    return A_inv

# test the inverse function
A = np.array([[1, 2], [3, 4]])
A_inv = inverse(A)
print(A_inv)


[[-2.   1. ]
 [ 1.5 -0.5]]


In [8]:
from scipy.optimize import linprog
from scipy.linalg import solve

def two_phase_simplex(c, A, b):
    """
    Solves a linear programming problem using the two-phase simplex method.

    Args:
        c: The objective function coefficients.
        A: The constraint matrix.
        b: The right-hand side of the constraints.

    Returns:
        The optimal solution and objective function value.
    """

    # Introduce auxiliary variables
    m, n = A.shape
    A_aug = np.hstack((A, np.eye(m)))
    c_aug = np.hstack((np.zeros(n), np.ones(m)))  # Extend c with ones for auxiliary variables

    # Solve the auxiliary problem
    res_phase1 = linprog(c_aug, A_eq=A_aug, b_eq=b, method="simplex")

    # Check if the original problem is feasible
    if res_phase1.status != 0:
        return None, np.inf

    # Phase II: Remove auxiliary variables
    B = res_phase1.x.argsort()[-m:][::-1]  # Basic variables
    N = list(set(range(n + m)) - set(B))  # Non-basic variables
    A_B = A_aug[:, B]
    A_N = A_aug[:, N]
    c_B = c_aug[B]
    c_N = c_aug[N]

    # Calculate initial tableau
    y = solve(A_B, b)
    z = c_B.dot(y)
    A_B_inv = np.linalg.inv(A_B)
    reduced_costs = c_N - c_B.dot(A_B_inv.dot(A_N))

    # Simplex iterations
    while any(reduced_costs < 0):
        # Find entering variable
        j = np.argmin(reduced_costs)
        d_N_j = np.zeros(n + m)
        d_N_j[j] = 1
        d_B_j = -A_B_inv.dot(A_N[:, j])

        # Find leaving variable
        theta = np.inf
        for i in range(m):
            if d_B_j[i] > 0:
                theta = min(theta, y[i] / d_B_j[i])
        if theta == np.inf:
            return None, np.inf  # Unbounded problem
        l = np.argmin(y[d_B_j > 0] / d_B_j[d_B_j > 0])

        # Update basis
        B[l] = N[j]
        N[j] = B[l]

        # Update tableau
        A_B = A_aug[:, B]
        A_N = A_aug[:, N]
        A_B_inv = np.linalg.inv(A_B)
        y = solve(A_B, b)
        z = c_B.dot(y)
        reduced_costs = c_N - c_B.dot(A_B_inv.dot(A_N))

    # Optimal solution
    x = np.zeros(n)
    x[B] = y
    return x[:n], z

# Example usage
c = np.array([1, 2, 3, 4])
A = np.array([[1, -1, 2, 0], [0, 1, -1, 2], [2, 0, 3, -1]])
b = np.array([2, 4, 2])

x, z = two_phase_simplex(c, A, b)
print("Optimal solution:", x)
print("Optimal objective value:", z)

Optimal solution: [0.         0.85714286 1.42857143 2.28571429]
Optimal objective value: 0.0


C:\Users\Z4N3R\AppData\Local\Temp\ipykernel_32100\1646530446.py:23: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res_phase1 = linprog(c_aug, A_eq=A_aug, b_eq=b, method="simplex")
